In [9]:
import os, sys
sys.path.append(os.path.dirname(os.getcwd()))

In [10]:
from utils import *
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

In [11]:
url = 'https://acl2019.org/schedule.acl2019.org/index.html'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

In [83]:
papers = []

for day in soup.find_all(class_='day-program'):
    for tr in day.find_all('div', class_='session-name'):
        if 'Oral Presentations' in tr.text:
            sessions = tr.parent.parent.parent.find_next_sibling('tr')
            papers_tr = sessions.find_next_sibling('tr').find_next_sibling('tr')
            for session, paper_column in zip(sessions.find_all('td'), papers_tr.find_all('td')):
                session_name = session.find(class_='conc-session-name')
                title_regex = r'Session \d.: (\D*)( \d)?'
                track = re.search(title_regex, session_name.text).group(1).strip()

                for paper in paper_column.find_all('a', class_='titlelink'):
                    title = paper.text
                    paper_id = paper.attrs['data-src'].split('-')[-1]
                    papers.append({'id': paper_id, 'title': title, 'area': track})

        elif tr.text.startswith('Poster Session'):
            papers_tr = tr.parent.parent.parent.find_next_sibling('tr')
            for subsession in papers_tr.find_all(class_='poster-sub-session'):
                subsession_name = subsession.find(class_='poster-session-name').text
                title_regex = r'Session \d.: (\D*)( \d)?'
                track = re.search(title_regex, subsession_name).group(1).strip()
                
                for paper in subsession.find_all('a', class_='titlelink'):
                    title = paper.text
                    paper_id = paper.attrs['data-src'].split('-')[-1]
                    papers.append({'id': paper_id, 'title': title, 'area': track})


df = pd.DataFrame(papers)
df          

,id,title,area
0,1793,One Time of Interaction May Not Be Enough: Go ...,Dialogue and Interactive Systems
1,1922,Incremental Transformer with Deliberation Deco...,Dialogue and Interactive Systems
2,2340,Improving Multi-turn Dialogue Modelling with U...,Dialogue and Interactive Systems
3,837,Do Neural Dialog Systems Use the Conversation ...,Dialogue and Interactive Systems
4,1693,Boosting Dialog Response Generation,Dialogue and Interactive Systems
...,...,...,...
676,889,Multimodal Transformer for Unaligned Multimoda...,"Vision, Robotics, Multimodal, Grounding and Sp..."
677,2155,"Show, Describe and Conclude: On Exploiting the...","Vision, Robotics, Multimodal, Grounding and Sp..."
678,384,Visual Story Post-Editing,"Vision, Robotics, Multimodal, Grounding and Sp..."
679,1891,Multimodal Abstractive Summarization for How2 ...,"Vision, Robotics, Multimodal, Grounding and Sp..."


In [84]:
set(df['area']) # there is no explicit interpretability area

{'Applications',
 'Bias in Language Processing',
 'Dialogue and Generation',
 'Dialogue and Interactive Systems',
 'Discourse and Pragmatics',
 'Document Analysis',
 'Evaluation',
 'Generation',
 'Information Extraction and Text Mining',
 'Linguistic Theories, Cognitive Modeling and Psycholinguistics',
 'Machine Learning',
 'Machine Translation',
 'Multidisciplinary',
 'Multilinguality',
 'Multilinguality and Morphology',
 'Phonology, Morphology and Word Segmentation',
 'Question Answering',
 'Resources and Evaluation',
 'Semantics',
 'Sentence-level Semantics',
 'Sentence-level semantics',
 'Sentiment Analysis and Argument Mining',
 'Social Media',
 'Summarization',
 'Tagging, Chunking, Syntax and Parsing',
 'Textual Inference and Other Areas of Semantics',
 'Vision, Robotics, Multimodal, Grounding and Speech',
 'Visual and Multimodal Question Answering',
 'Word-level Semantics'}

In [85]:
df['interpretability'] = None
df['doi'] = df['title'].apply(get_acl_anthology_doi)
df['source'] = 'ACL2019'
df

,id,title,area,interpretability,doi,source
0,1793,One Time of Interaction May Not Be Enough: Go ...,Dialogue and Interactive Systems,None,10.18653/v1/P19-1001,ACL2019
1,1922,Incremental Transformer with Deliberation Deco...,Dialogue and Interactive Systems,None,10.18653/v1/P19-1002,ACL2019
2,2340,Improving Multi-turn Dialogue Modelling with U...,Dialogue and Interactive Systems,None,10.18653/v1/P19-1003,ACL2019
3,837,Do Neural Dialog Systems Use the Conversation ...,Dialogue and Interactive Systems,None,10.18653/v1/P19-1004,ACL2019
4,1693,Boosting Dialog Response Generation,Dialogue and Interactive Systems,None,10.18653/v1/P19-1005,ACL2019
...,...,...,...,...,...,...
676,889,Multimodal Transformer for Unaligned Multimoda...,"Vision, Robotics, Multimodal, Grounding and Sp...",None,10.18653/v1/P19-1656,ACL2019
677,2155,"Show, Describe and Conclude: On Exploiting the...","Vision, Robotics, Multimodal, Grounding and Sp...",None,10.18653/v1/P19-1657,ACL2019
678,384,Visual Story Post-Editing,"Vision, Robotics, Multimodal, Grounding and Sp...",None,10.18653/v1/P19-1658,ACL2019
679,1891,Multimodal Abstractive Summarization for How2 ...,"Vision, Robotics, Multimodal, Grounding and Sp...",None,10.18653/v1/P19-1659,ACL2019


In [86]:
df['area'].value_counts()

area
Machine Learning                                                 64
Information Extraction and Text Mining                           53
Dialogue and Interactive Systems                                 51
Machine Translation                                              49
Question Answering                                               41
Generation                                                       38
Applications                                                     33
Sentiment Analysis and Argument Mining                           33
Tagging, Chunking, Syntax and Parsing                            30
Word-level Semantics                                             27
Vision, Robotics, Multimodal, Grounding and Speech               25
Resources and Evaluation                                         25
Social Media                                                     25
Summarization                                                    24
Multilinguality                            

# Saving the results

In [87]:
previous_df = pd.read_csv('../data/parsed_data.csv', sep=",", index_col=0)
previous_df

,id,title,area,interpretability,doi,source
0,187,Probabilistic FastText for Multi-Sense Word Em...,Word Semantics,NaN,10.18653/v1/P18-1001,ACL2018
1,1520,A La Carte Embedding: Cheap but Effective Indu...,Word Semantics,NaN,10.18653/v1/P18-1002,ACL2018
2,707,Unsupervised Learning of Distributional Relati...,Word Semantics,NaN,10.18653/v1/P18-1003,ACL2018
3,1553,Explicit Retrofitting of Distributional Word V...,Word Semantics,NaN,10.18653/v1/P18-1004,ACL2018
4,76,Unsupervised Neural Machine Translation with W...,Machine Translation,NaN,10.18653/v1/P18-1005,ACL2018
...,...,...,...,...,...,...
9019,s1835,On Extractive and Abstractive Neural Document ...,Summarization,False,10.18653/v1/2020.emnlp-main.748,EMNLP2020
9020,s2125,Multi-Fact Correction in Abstractive Text Summ...,Summarization,False,10.18653/v1/2020.emnlp-main.749,EMNLP2020
9021,s2506,Evaluating the Factual Consistency of Abstract...,Summarization,False,10.18653/v1/2020.emnlp-main.750,EMNLP2020
9022,s3552,Re-evaluating Evaluation in Text Summarization,Summarization,False,10.18653/v1/2020.emnlp-main.751,EMNLP2020


In [88]:
# we'll delete previous rows from the conference
previous_df = previous_df[previous_df['source'] != 'ACL2019']

In [89]:
updated_df = pd.concat([previous_df, df], ignore_index=True)
updated_df

,id,title,area,interpretability,doi,source
0,187,Probabilistic FastText for Multi-Sense Word Em...,Word Semantics,NaN,10.18653/v1/P18-1001,ACL2018
1,1520,A La Carte Embedding: Cheap but Effective Indu...,Word Semantics,NaN,10.18653/v1/P18-1002,ACL2018
2,707,Unsupervised Learning of Distributional Relati...,Word Semantics,NaN,10.18653/v1/P18-1003,ACL2018
3,1553,Explicit Retrofitting of Distributional Word V...,Word Semantics,NaN,10.18653/v1/P18-1004,ACL2018
4,76,Unsupervised Neural Machine Translation with W...,Machine Translation,NaN,10.18653/v1/P18-1005,ACL2018
...,...,...,...,...,...,...
9299,889,Multimodal Transformer for Unaligned Multimoda...,"Vision, Robotics, Multimodal, Grounding and Sp...",None,10.18653/v1/P19-1656,ACL2019
9300,2155,"Show, Describe and Conclude: On Exploiting the...","Vision, Robotics, Multimodal, Grounding and Sp...",None,10.18653/v1/P19-1657,ACL2019
9301,384,Visual Story Post-Editing,"Vision, Robotics, Multimodal, Grounding and Sp...",None,10.18653/v1/P19-1658,ACL2019
9302,1891,Multimodal Abstractive Summarization for How2 ...,"Vision, Robotics, Multimodal, Grounding and Sp...",None,10.18653/v1/P19-1659,ACL2019


In [90]:
updated_df.to_csv('../data/parsed_data.csv')